# Paso 1 - Cargar y procesar dataset

En este caso, agregamos una columna que es el continente del país. El ejercicio será entrenar un modelo que pueda identificar en qué país está el Airbnb según sus datos no espaciales.

In [1]:
import pandas as pd

df = pd.read_csv("Airbnb_Locations.csv")

continente = {
    'Australia': 'Oceania',
    'Thailand': 'Asia',
    'Brazil': 'Americas',
    'France': 'Europe',
    'South Africa': 'Africa',
    'Italy': 'Europe',
    'Mexico': 'Americas',
    'Turkey': 'Europe',
    'United States': 'Americas',
    'China': 'Asia'
}

df['continente'] = df['pais'].map(continente)

columnas_no_deseadas = ['pais', 'anfitrión/a','latitud','longitud', 'ciudad']

# Eliminar columnas no deseadas
df = df.drop(columnas_no_deseadas, axis=1, inplace=False)
df.head()

,tiempo_respuesta,es_superhost,tipo_propiedad,capacidad,puntaje_promedio_comunicación,puntaje_promedio_localización,servicio_tv_cable,servicio_aire_acondicionado,continente
0,dentro de una hora,0,pieza privada en casa,2,10,10,1,0,Oceania
1,dentro de una hora,0,apartamento completo,4,10,10,0,1,Asia
2,dentro de una hora,0,pieza privada en departamento,2,10,10,1,1,Americas
3,dentro de una hora,1,condominio completo,2,10,10,1,1,Asia
4,dentro de una hora,0,apartamento completo,6,10,10,1,0,Europe


# Paso 2 - Guardar Pipeline

Vamos a dividir los datos en X, Y, y luego construir un pipeline que preprocesa los datos categóricos + entregnar un Random Forests.

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import joblib

X = df.drop('continente', axis=1)
y = df['continente']


# Separar las columnas numéricas y categóricas
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Definir los transformers para cada tipo de dato
numeric_transformer = 'passthrough'
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Crear un preprocesador con ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Definir el modelo
model = RandomForestClassifier()

# Crear el pipeline completo
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', model)])

# Entrenar el pipeline
pipeline.fit(X, y)

# Guardar el pipeline entrenado
joblib.dump(pipeline, 'pipeline_model.pkl')

['pipeline_model.pkl']

### Probar pipeline

In [5]:
# Cargar pipeline
pipeline_loaded = joblib.load('pipeline_model.pkl')

def predict(info):
    # Dar las columnas como corresponde
    columns = ['tiempo_respuesta', 'es_superhost', 'tipo_propiedad', 'capacidad',
        'puntaje_promedio_comunicación', 'puntaje_promedio_localización',
        'servicio_tv_cable', 'servicio_aire_acondicionado']

    # Construir DataFrame con los datos y columnas
    df_test = pd.DataFrame([info], columns=columns)

    # Clasificar obtienendo la probabilidad por clase
    predictions = pipeline_loaded.predict_proba(df_test)
    return {
        "classes": pipeline_loaded.classes_,
        "probabilities": predictions[0],
        "result": pipeline_loaded.predict(df_test)[0]
    }

datos_prueba = ['dentro de una hora', 0, 'pieza privada en casa', 2, 10, 10, 1, 0]
predict(datos_prueba)

{'classes': array(['Africa', 'Americas', 'Asia', 'Europe', 'Oceania'], dtype=object),
 'probabilities': array([0.26103175, 0.26888095, 0.        , 0.        , 0.4700873 ]),
 'result': 'Oceania'}